In this notebook, we are just reading all the feature vectors previously saved to see how they perform individually or when combined in different ways. The results at the end are with just SURF

In [ ]:
                                                                                                                                                                                                                                                                                                    from google.colab import drive
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler as ss
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [ ]:
drive.mount('/drive') 

Mounted at /drive


In [ ]:
trainDataORB = pd.read_csv('/drive/My Drive/CAD_1_1/rawORB_Train.csv')
trainDataSIFT = pd.read_csv('/drive/My Drive/CAD_1_1/rawSIFT_Train.csv')
trainDataSURF = pd.read_csv('/drive/My Drive/CAD_1_1/rawSURF_Train.csv')

valDataORB = pd.read_csv('/drive/My Drive/CAD_1_1/rawORB_val.csv')
valDataSIFT = pd.read_csv('/drive/My Drive/CAD_1_1/rawSIFT_val.csv')
valDataSURF = pd.read_csv('/drive/My Drive/CAD_1_1/rawSURF_val.csv')

trainGoodLBP =  pd.read_csv('/drive/My Drive/CAD_1_1/goodLBP_Train.csv')
valGoodLBP = pd.read_csv('/drive/My Drive/CAD_1_1/goodLBP_val.csv')

trainGoodglcm =  pd.read_csv('/drive/My Drive/CAD_1_1/goodglcm3_Train.csv')
valGoodglcm = pd.read_csv('/drive/My Drive/CAD_1_1/goodglcm3_val.csv')

trainGoodcolor =  pd.read_csv('/drive/My Drive/CAD_1_1/goodcolor_Train.csv')
valGoodcolor = pd.read_csv('/drive/My Drive/CAD_1_1/goodcolor_val.csv')

trainDataRGBLBPGLCM = pd.read_csv('/drive/My Drive/CAD_1_1/raw_RGB_LBP_GLCM_Train_hairRemoved.csv')
valDataRGBLBPGLCM = pd.read_csv('/drive/My Drive/CAD_1_1/raw_RGB_LBP_GLCM_val_hairRemoved.csv')

trainLBPRGB =  pd.read_csv('/drive/My Drive/CAD_1_1/LBPrgb_Train.csv')
valLBPRGB = pd.read_csv('/drive/My Drive/CAD_1_1/LBPrgb_val.csv')

trainColourFeatures = pd.read_csv('/drive/My Drive/CAD_1_1/colourFeatures_Train_hairRemoved.csv')
valColourFeatures = pd.read_csv('/drive/My Drive/CAD_1_1/colourFeatures_val_hairRemoved.csv')

In [ ]:
trainDataShuffle=trainDataSURF.sample(frac=1)
valDataShuffle=valDataSURF.sample(frac=1)

In [ ]:
trainDataShuffle.shape

(4800, 21)

In [ ]:
valDataShuffle.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,labels
739,0.033333,0.077778,0.000000,0.092593,0.062963,0.037037,0.081481,0.118519,0.035185,0.064815,0.0,0.081481,0.070370,0.059259,0.042593,0.046296,0.059259,0.024074,0.009259,0.003704,0
1016,0.018868,0.056604,0.018868,0.056604,0.009434,0.009434,0.198113,0.056604,0.028302,0.066038,0.0,0.169811,0.066038,0.056604,0.094340,0.018868,0.075472,0.000000,0.000000,0.000000,0
20,0.021583,0.021583,0.000000,0.172662,0.107914,0.028777,0.057554,0.071942,0.050360,0.057554,0.0,0.093525,0.021583,0.079137,0.086331,0.057554,0.057554,0.000000,0.000000,0.014388,1
964,0.023179,0.019868,0.000000,0.092715,0.079470,0.013245,0.013245,0.079470,0.082781,0.046358,0.0,0.086093,0.036424,0.026490,0.039735,0.109272,0.178808,0.059603,0.013245,0.000000,0
974,0.000000,0.040816,0.000000,0.183673,0.163265,0.020408,0.020408,0.020408,0.020408,0.081633,0.0,0.000000,0.061224,0.000000,0.163265,0.020408,0.183673,0.020408,0.000000,0.000000,0


In [ ]:
rndst=RandomForestClassifier(n_estimators=100)
rndst.fit(trainDataShuffle.drop(["labels"],axis=1), trainDataShuffle['labels'])
rndst.score(valDataShuffle.drop(["labels"],axis=1),valDataShuffle['labels'])

0.75

In [ ]:
best_svc = SVC()
best_svc.fit(trainDataShuffle.drop(["labels"],axis=1), trainDataShuffle['labels'])
best_svc.score(valDataShuffle.drop(["labels"],axis=1),valDataShuffle['labels'])

0.7516666666666667

In [ ]:
scaler = ss()

scaledTrain = scaler.fit_transform(trainDataShuffle.drop(["labels"],axis=1))
scaledTrain.shape
scaledVal = scaler.fit_transform(valDataShuffle.drop(["labels"],axis=1))


rndst.fit(scaledTrain, trainDataShuffle['labels'])
rndst.score(scaledVal,valDataShuffle['labels'])

0.8641666666666666

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
svc=SVC(kernel='linear')
svc.fit(scaledTrain, trainDataShuffle['labels'])
predict=svc.predict(scaledVal)
#rndst.score(predict,valData['labels'])
print(metrics.accuracy_score(valDataShuffle['labels'],predict))

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(valDataShuffle['labels'],predict, labels=[0,1]))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(valDataShuffle['labels'],predict, labels=[0,1]))

In [ ]:
#PCA maybe

# from sklearn.decomposition import PCA
# pca = PCA(n_components=1250)
# pca.fit(trainDataShuffle.drop(["labels"],axis=1))
# trainDataPCAed = pca.transform(scaledTrain)
# valDataPCAed = pca.transform(scaledVal)

# svc2=RandomForestClassifier(n_estimators=100)
# svc2.fit(trainDataPCAed, trainDataShuffle['labels'])
# svc2.score(valDataPCAed,valDataShuffle['labels'])

# #pca is bad

0.8116666666666666

###GridSearch

In [ ]:
X_train = scaledTrain
X_test = scaledVal
y_train = trainDataShuffle["labels"]
y_test = valDataShuffle['labels']

In [ ]:
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgbm

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
SEED = 123
cross_valid_scores = {}
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, make_scorer

Decision Tree Classifier


In [ ]:
%%time

parameters = {
    "max_depth": [3, 5, 21,51,101,181,300,450, 600],
}

model_desicion_tree = DecisionTreeClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_desicion_tree = GridSearchCV(
    model_desicion_tree, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='f1_macro',
    verbose = 10,
)

model_desicion_tree.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_desicion_tree.best_params_}')
print(
    f'Mean cross-validated  score of the best_estimator: ' + \
    f'{model_desicion_tree.best_score_:.3f}'
)
cross_valid_scores['desicion_tree'] = model_desicion_tree.best_score_
print('-----')

In [ ]:

res = pd.concat([pd.DataFrame(model_desicion_tree.cv_results_["params"]),pd.DataFrame(model_desicion_tree.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_desicion_tree.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_desicion_tree.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred))
res

# Random Forest Classifier

In [ ]:

%%time
parameters = {
    "n_estimators": [ 1800,2000, 2500], 
    "max_depth": [  9, 11,13],
}

model_random_forest = RandomForestClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_random_forest = GridSearchCV(
    model_random_forest, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_random_forest.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_random_forest.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_random_forest.best_score_:.3f}'
)
cross_valid_scores['random_forest'] = model_random_forest.best_score_
print('-----')

In [ ]:

res = pd.concat([pd.DataFrame(model_random_forest.cv_results_["params"]),pd.DataFrame(model_random_forest.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_random_forest.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_random_forest.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred))
res

ADABOOST

In [ ]:
%%time
parameters = {
    "n_estimators": [ 10,100, 300], 
    "learning_rate": [ 0.1, 0.5, 1.1, 1.2, 1.3 ],
}

model_adaboost = AdaBoostClassifier(
    random_state=SEED,
)

model_adaboost = GridSearchCV(
    model_adaboost, 
    parameters, 
    cv=StratifiedKFold(n_splits=2,shuffle = True, random_state= SEED),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_adaboost.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_adaboost.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_adaboost.best_score_:.3f}'
)
cross_valid_scores['ada_boost'] = model_adaboost.best_score_
print('-----')

In [ ]:

res = pd.concat([pd.DataFrame(model_adaboost.cv_results_["params"]),pd.DataFrame(model_adaboost.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_adaboost.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_adaboost.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred))
res

SVC

In [ ]:
%%time
parameters = {
    "C": [ 20, 50,100],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
}

model_svc = SVC(
    random_state=SEED,
    class_weight="balanced",
    probability=True,
)

model_svc = GridSearchCV(
    model_svc, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_svc.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_svc.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_svc.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_svc.best_score_
print('-----')

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] C=20, kernel=linear .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................. C=20, kernel=linear, score=0.863, total=20.6min
[CV] C=20, kernel=linear .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 20.6min remaining:    0.0s


In [ ]:
res = pd.concat([pd.DataFrame(model_svc.cv_results_["params"]),pd.DataFrame(model_svc.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_svc.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_svc.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred))
res 

GaussianNB

In [ ]:
%%time
parameters =  {'var_smoothing': np.logspace(0,-9, num=100)}

model_gaussianNB = GaussianNB()

model_gaussianNB = GridSearchCV(
    model_gaussianNB, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
    n_jobs = -1
)

model_gaussianNB.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_gaussianNB.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gaussianNB.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_gaussianNB.best_score_
print('-----')

In [ ]:
res = pd.concat([pd.DataFrame(model_gaussianNB.cv_results_["params"]),pd.DataFrame(model_gaussianNB.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_gaussianNB.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_gaussianNB.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred))
res

KNeighborsClassifier


In [ ]:
%%time
parameters =    {
        'n_neighbors': [3, 5, 7, 9] # usually odd numbers
    }

model_KNN =  KNeighborsClassifier()

model_KNN = GridSearchCV(
    model_KNN, 
    parameters, 
    cv=StratifiedKFold(n_splits=2),
    scoring='roc_auc_ovr',
    verbose = 10,
    n_jobs = -1
 
)

model_KNN.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_KNN.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_KNN.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_KNN.best_score_
print('-----')

res = pd.concat([pd.DataFrame(model_KNN.cv_results_["params"]),pd.DataFrame(model_KNN.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_KNN.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_KNN.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred))
res